Let's load the dataset.

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

In [28]:
dataset = pd.read_csv('play_dataset.csv')
dataset = dataset.drop(columns = 'Unnamed: 0')
dataset

,tipologia,genere,zona,bagno,cucina,salotto,balcone,range euro
0,doppia,F,Murri,2,1,0,0,200-225
1,doppia,F,Marconi,2,1,1,0,225-250
2,doppia,M/F,Bolognina,1,1,1,0,250-275
3,singola,F,S.Vitale,1,1,0,1,400-425
4,singola,F,Irnerio,1,1,0,1,500-525
...,...,...,...,...,...,...,...,...
195,doppia,M,Irnerio,1,1,0,0,200-225
196,doppia,M/F,Bolognina,2,1,1,1,225-250
197,doppia,M/F,Bolognina,1,1,1,1,225-250
198,singola,M/F,San Donato,1,1,0,0,350-375


In [29]:
dummy_dataset = pd.get_dummies(dataset, columns = ['tipologia', 'genere', 'zona'])
columns = list(dummy_dataset.columns.values)
columns.pop(columns.index('range euro')) 
dummy_dataset = dummy_dataset[columns+['range euro']]
dummy_dataset

,bagno,cucina,salotto,balcone,tipologia_doppia,tipologia_singola,genere_F,genere_M,genere_M/F,zona_Barca,...,zona_Malpighi,zona_Marconi,zona_Mazzini,zona_Murri,zona_S.Ruffillo,zona_S.Viola,zona_S.Vitale,zona_Saffi,zona_San Donato,range euro
0,2,1,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,200-225
1,2,1,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,225-250
2,1,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,250-275
3,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,400-425
4,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,500-525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,200-225
196,2,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,225-250
197,1,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,225-250
198,1,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,350-375


In [30]:
array = dummy_dataset.values
X_enc = array[:,0:27]
Y = array[:,27]

In [31]:
le = preprocessing.LabelEncoder()
le.fit_transform(['175-200', '200-225', '225-250', '250-275', '275-300', '300-325', '325-350', '350-375',
                  '375-400', '400-425', '425-450', '450-475', '475-500', '500-525', '525-550', '550-575', 
                 '575-600', '600-625', '625-650', '650-675', '675-700'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [32]:
Y_enc = le.transform(Y)

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X_enc, Y_enc, test_size = 0.2)

Let's play.